## Lens-focused Laguerre-Gauss beam as per GLMT: <br></br>A Finite Series sketch

In [408]:
# Imports
import numpy as np               # Ye olde numpy
import scipy.special as sp       # Special functions
import matplotlib.pyplot as plt  # Plots results
import mpmath                    # Arbitrary precision
import scipy.integrate as intg   # Numerical integration

import inspect                   # Maybe remove later (decorator things)

In [409]:
# LFLG beam parameters:
p, l = 0, 2
alpha, beta = 1, 0
r_focus = 1E-3
wavelength = 6328E-9
na = .95
w0 = 3.2 * wavelength

k = 2 * np.pi / wavelength
s = 1 / k / w0

parameters = {
    "p": p,
    "l": l,
    "alpha": alpha,
    "beta": beta,
    "r_focus": r_focus,
    "wavelength": wavelength,
    "na": na,
    "w0": w0,
    "delta": 1,
}

In [407]:
# Memoization decorator
def memoized(func):
    CACHE = {}
    def memoized_func(*args, **kwargs):
        kwvals = [val for _, val in kwargs.items()]
        if (*args, *kwvals) not in CACHE:
            value = func(*args, **kwargs)
            CACHE[(*args, *kwvals)] = value
            return value
        return CACHE[(*args, *kwvals)]
    return memoized_func

In [404]:
from IPython.display import Javascript # some cool shenanigans
display(Javascript('IPython.notebook.execute_cells_above()'))
display(Javascript('IPython.notebook.execute_cells_above()'))

class LGBeamFS:
    def __init__(self, **kwargs):
        self.__dict__.update(parameters)
        self.__dict__.update(kwargs)
        
        self.k = 2 * np.pi / self.wavelength
        self.s = 1 / self.k / self.w0
        if 'delta' not in parameters and 'delta' not in kwargs:
            self.delta = 1
    
    def a(self, eta):
        return eta * self.r_focus * self.s * mpmath.sqrt(2)
    
    def lens_integrand(self, eta, n, p, kind='A'):
        pm = 1 if kind == 'A' else -1
        if p == 0:
            lfactor = 1
        elif p >= 1:
            lfactor = mpmath.laguerre(p - 1, self.l, self.a(eta) ** 2) \
                    * self.a(eta) ** 2
        else:
            return 0
        
        res = mpmath.power(eta, np.abs(l + 1)) \
            / mpmath.power(self.k ** 2 - eta ** 2, .25) \
            * lfactor * mpmath.exp(-self.a(eta) ** 2 / 2) \
            * (1 + pm * mpmath.sqrt(self.k ** 2 - eta ** 2) / self.k) \
            * mpmath.power(eta / k, n - 1)
        return res
        
        return res
    
    def F(self, j, u):
        return (-1) ** ((j - u) / 2) \
             / mpmath.factorial((j - u) / 2) / mpmath.factorial((j + u) / 2)
    
    @memoized
    def lens_integral(self, n, p, kind='A'):
        max_eta = self.k * self.na * self.delta
        return intg.quad(self.lens_integrand, 0, max_eta, args=(n, p, kind))[0]
    
    @memoized
    def maclaurin(self, j, p):
        lpart, rpart = 1, 1
        premul = 1j * np.sqrt(2 / np.pi) * (self.alpha - 1j * self.beta) \
               / mpmath.power(2, j)
        if j <= l:
            lpart = 0
            if j <= l - 2:
                rpart = 0

        if rpart != 0:
            rpart = self.F(j - 1, self.l - 2) \
                  * self.lens_integral(j, p, kind='B')
            if lpart != 0:
                lpart = self.F(j - 1, self.l) \
                      * self.lens_integral(j, 0, kind='A')
        if p == 1:
            if rpart != 0:
                rpart = self.F(j - 1, self.l - 2) \
                      * ((l + 1) * self.lens_integral(j, 0, kind='B') \
                        - self.lens_integral(j, 1, kind='B'))
                if lpart != 0:
                    lpart = self.F(j - 1, self.l) \
                          * ((l + 1) * self.lens_integral(j, 0, kind='A') \
                            - self.lens_integral(j, 1, kind='A'))
        elif p != 0:
            premul = premul / p
            return ((2 * p - 1 + self.l) / p * self.maclaurin(j, p - 1) \
                    - (p - 1 + self.l) / p * self.maclaurin(j, p - 2) \
                    - premul * (lpart + rpart)
                   )
            
        return premul * (lpart + rpart)
    
    def nconv_tmbsc_lm1(self, n):
        premul = 1j ** n * (-1) ** ((n - self.l + 1) / 2) \
               * mpmath.power(2, n + 1.5 - self.l) * np.sqrt(np.pi) \
               / mpmath.gammaprod([(n - l + 3) / 2], [(n + l) / 2])
    
        fs = 0
        q = 0
        while q <= n / 2:
            inc = mpmath.power(2, -2 * q) \
                * mpmath.gammaprod([.5 + n - q], [q + 1]) \
                * self.maclaurin(n - 2 * q, p)
            q += 1
        return premul * fs
    
    def tmbsc_lm1(self, n):
        return (-1) ** (n + 1) * mpmath.conj(self.nconv_tmbsc_lm1(n))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Test:

In [410]:
lgbeam = LGBeamFS(**parameters)
print(lgbeam.tmbsc_lm1(l + 3))

(0.0 + 0.0j)


### Thought process:

We are need the following function represented by $\texttt{gammaprod}$ in the $\texttt{mpmath}$ package:

$$ \texttt{gammaprod}(\textbf{u}, \textbf{v})  = \frac{\Gamma(u_1) \cdot \Gamma(u_2) \cdot\ldots\cdot \Gamma(u_n)}{\Gamma(v_1) \cdot \Gamma(v_2) \cdot\ldots\cdot \Gamma(v_m)}$$

For more info see <a href="http://mpmath.org/doc/current/functions/gamma.html#gammaprod">entry</a> in $\texttt{mpmath}$ documentation.

Also note that here we will compute harmonic modes in the $N$-convention, meaning we must not forget to convert the BSCs to the $P$-convention via:

$$g_{n, TX}^m (P) = (-1)^{n + 1} g_{n, TX}^{-m} (N)^*.$$

In [33]:
# N-convention _ TM LFLG BSC _ m = l - 1 finite series
def nconv_tmlflgbsc_lm1fs(n, m, **parameters):
    pass

# [P-convention _] TM LFLG BSC _ m = l - 1 finite series
def tmlflgbsc_lm1fs(n, m, **parameters):
    return (-1 ** (n + 1)) * nconv_tmlflgbsc_lm1fs(n, -m, **parameters)

#### Case (ii): $m = l - 1, l > 0$
$$    \left[ g_{n, TM}^{l-1}(N) \right]_p = i^n \frac{\sqrt{\pi}}{2^{l}} (-1)^{\frac{n - l + 1}{2}} \frac{\Gamma\left( \frac{n - l + 3}{2} \right)}{\Gamma\left( \frac{n + l}{2} \right)}
    \sum_{q=0}^{\leq n / 2} 2^{\frac{1}{2}+n - 2q} \frac{\Gamma\left(\frac{1}{2}+n-q\right)}{q!} \left[ b_{n-2q} \right]_p$$

### Doodles

Trying to figure out some stuff about decorators...

In [64]:
def zero(*args, **kwargs):
    return 0
def epsilonized(inname, outname):

    def find_arg_with_name(name, arglist, func_args=[], func_kwargs={}):
        value = fkwargs.get(name)
        if value is None:
            return func_args[arglist.index(name)]
        return value

    def decorator_eps(func):
        def epsilonized_func(*args, **kwargs):
            j = find_arg_with_name(
                inname, inspect.getfullargspec(func),
                func_args=args, func_kwargs=kwargs
            )
            if j in A:
                value = 0
            else:
                value = func(*args, **kwargs)
            return value
        return epsilonized_func
    return decorator_eps

Quadrature integration from $\texttt{scipy.integrate}$

In [134]:
def foo(x, a, b=0):
    return a * x + b

intg.quad(foo, 0, 1, args=(1, 1)) # args are processed in order; even kwargs

(1.5, 1.6653345369377348e-14)

In [332]:
d = {1: 2, 3: 4}
tuples = d.items()
print(tuples)

dict_items([(1, 2), (3, 4)])


In [330]:
kw = {1: 2}
kw.items()

dict_items([(1, 2)])